In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
from collections import Counter
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import file_utils
import db_utils
import config
#set working directory
os.chdir(config.working_dir)

In [5]:
# db credentials authentication

In [6]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [7]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [8]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_data = pd.read_sql_query('select * from cs_retail_loan_cust', con)
con.close()

In [9]:
# get for retail loan customers only and insert into database

for i, f in enumerate(file_utils.get_files(r"N:\10. Executive\Original\ETL\in_branch_trxn")):
    print i+1, '. ', f
    cur_df = file_utils.unzip(f)
    cur_df = cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']].merge(
        cur_df, on=['BRA_CODE','CUS_NUM'], how='inner').fillna(0)
        
    if_exists_var = 'append'
    if i==0:
        if_exists_var = 'replace'
    
    cur_df['REFERENCE_DATE'] = pd.to_datetime(cur_df['REFERENCE_DATE'], format="%d%b%Y")
    cur_df['CTRL_DATE'] = pd.to_datetime(cur_df['CTRL_DATE'], format="%d%b%Y")
    
    conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
    cur_df.to_sql('cs_cash_wdr', conn, index=False, if_exists=if_exists_var, chunksize=10000,
                                                                      dtype=db_utils.sqlcol(cur_df))
    del cur_df

1 .  N:\10. Executive\Original\ETL\in_branch_trxn\17Feb2020.csv.gz
2 .  N:\10. Executive\Original\ETL\in_branch_trxn\28Feb2018.csv.gz
3 .  N:\10. Executive\Original\ETL\in_branch_trxn\28Feb2019.csv.gz
4 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Apr2018.csv.gz
5 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Apr2019.csv.gz
6 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Jun2018.csv.gz
7 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Jun2019.csv.gz
8 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Nov2018.csv.gz
9 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Nov2019.csv.gz
10 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Sep2018.csv.gz
11 .  N:\10. Executive\Original\ETL\in_branch_trxn\30Sep2019.csv.gz
12 .  N:\10. Executive\Original\ETL\in_branch_trxn\31Aug2018.csv.gz
13 .  N:\10. Executive\Original\ETL\in_branch_trxn\31Aug2019.csv.gz
14 .  N:\10. Executive\Original\ETL\in_branch_trxn\31Dec2018.csv.gz
15 .  N:\10. Executive\Original\ETL\in_branch_trxn\31Dec2

In [10]:
con = cx_Oracle.connect(conn_string)
df_retail_loan_cust_in_branch_info = pd.read_sql_query(r'''
select a.bra_code, a.cus_num, nvl(sum(cash_wdr_cnt)+sum(fastrack_cash_wdr_cnt),0) cash_wdr_cnt,
nvl(sum(clearing_wdr_cnt),0) cheques_issued
from cs_retail_loan_cust a
left join cs_cash_wdr b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
and b.reference_date between trunc(add_months(a.date_open, -12),'month') and trunc(add_months(a.date_open, -1),'month')
group by a.bra_code, a.cus_num
''', con)
con.close()

In [11]:
print len(df_retail_loan_cust_in_branch_info)
# old: 12562

154724


In [12]:
# df_retail_loan_cust_in_branch_info[df_retail_loan_cust_in_branch_info.CHEQUES_ISSUED>0].IS_BAD.value_counts()

In [13]:
df_retail_loan_cust_in_branch_info.to_csv('cs_retail_loan_cust_in_branch_info.csv', index=False)